# Figure 5 & 6: Size Averaged $\lambda$ dependence, Size Distributions

In [ ]:
%run header.py

In [ ]:
d      = np.load(opacity.get_datafile('default_opacities_smooth.npz'))
a      = d['a']
lam    = d['lam']
k_abs  = d['k_abs']
k_sca  = d['k_sca']

In [ ]:
# Download one of the Weingartner & Draine 2001 dust models for comparison

filename = 'kext_albedo_WD_MW_3.1_60'
if not os.path.isfile(filename):
    from urllib.request import urlretrieve
    print('Downloading file \'{}\' ... '.format(filename), end='')
    urlretrieve(
        'ftp://ftp.astro.princeton.edu/draine/dust/mix/{}'.format(filename),
        filename=filename)
    print('Done!')

with open(filename) as f:
    while not f.readline().startswith('--------'):
        pass
    lam_dr,_,_,_,k_dr = np.loadtxt(f).T

In [ ]:
# a 1g-normalized size distribution (bin-integrated) up to 1 mm

s = a**0.5
s[a > 0.1] = 0
s= s / s.sum()

# size average the absorption opacities

k_powerlaw  = (k_abs * s[:,None]).sum(0)

Create a coagulation-fragmentation size distribution with similar upper size limit.

In [ ]:
r = c.au.cgs.value          # position in the disk in cm
T = 200.                    # temperature in K
sigma_g = 200.              # gas surface density [g/cm^2]
d2g = 0.01                  # dust-to-gas ratio
rho_s = 1.6686              # material density [g/cm^3]
M_star = c.M_sun.cgs.value  # stellar mass [g]
v_frag = 100.               # fragmentation velocity [cm/s]
alpha = 4e-4                # turbulence parameter

m = 4 * np.pi / 3 * rho_s * a**3  # mass grid

# create the size distribution, FIRST: the fortran code

fit, a_01, a_12, a_l, a_p, a_r, a_sett = opacity.distribution(1.8, T, alpha, sigma_g, sigma_g * d2g, rho_s, m, a, M_star, r, v_frag)

# SECOND: a simplified version. Need to convert to bin-integrated values.

fit2, a_f = opacity.get_B11S_fit(T, a, r=r, sigma_g=sigma_g, d2g=d2g, rho_s=rho_s, M_star=M_star, v_frag=v_frag, alpha=alpha)
fit2 = fit2 * np.diff(np.hstack((a[0]**2 / a[1], a)))
fit2 = fit2 / fit2.sum() * sigma_g * d2g

# plot the size distribution

fig, ax = plt.subplots(figsize=(3.5, 3.5 * 0.66))
ax.loglog(a, s * sigma_g * d2g, label='power-law distribution')
ax.loglog(a, fit, label='B11')
ax.loglog(a, fit2, label='B11S')
ax.set_xlim(1e-5, 2e-1)
ax.set_ylim(1e-4, 2e-1)
ax.set_xlabel('particle size [cm]')
ax.set_ylabel('$\sigma$ [g cm$^{-2}$]')
ax.legend(fontsize='small')

fig.subplots_adjust(
    bottom=0.17, top=0.95,
    left=0.14, right=0.97
    )
fig.savefig('figures/fig5_size_distri.pdf')

In [ ]:
# size average the absorption opacities
s2 = fit / fit.sum()
k_fit  = (k_abs * s2[:,None]).sum(0)

s3 = opacity.get_B11S_fit(T, a)[0]
s3 *= a
s3 = s3 / s3.sum()
k_fit2  = (k_abs * s3[:,None]).sum(0)

In [ ]:
# where to measure the reference value
lam_obs = 0.087

# load the D'Alessio opacity
d2g = sum([0.0056, 0.0034, 0.000768, 0.0041])
data_d01 = np.loadtxt(opacity.get_datafile('kappa_D01_T100K_p3.5_amax1mm.csv'))
lam_d01 = 10.**data_d01[:,0] * 1e-4
kap_d01 = 10.**(data_d01[:,1]) / d2g

In [ ]:
# the Beckwith 1990 law

kb = 3.446 * (lam / 0.087)**(-1)  # Beckwith 1990

# the opacities from Andrews et al. 2009

la, ka = np.loadtxt(opacity.get_datafile('andrews2009.dat')).T

# now the plot

f, ax = plt.subplots(figsize=(7, 7 * 0.64))

ax.loglog(1e-4*lam_dr, k_dr,       'k',   zorder=-100, alpha=0.5, label='Weingartner & Draine 2001')
ax.loglog(lam,         kb,         'k--', zorder=-100, alpha=0.5, label='Beckwith et al. 1990')
ax.loglog(1e-4*la,     ka,         'k--', zorder=-100, alpha=1.0, label='Andrews et al. 2009')
ax.loglog(lam_d01,     kap_d01,    'k:',  zorder=-100, alpha=1.0, label='D\'Alessio et al. 2001')
l1, = ax.loglog(lam,   k_powerlaw,        zorder=-100, alpha=1.0, label='DSHARP, power-law')
l2, = ax.loglog(lam,   k_fit,             zorder=-100, alpha=1.0, label='DSHARP, B11', ls='--')
l2, = ax.loglog(lam,   k_fit2,            zorder=-100, alpha=1.0, label='DSHARP, B11S', ls='--')

ax.legend(loc=3, fontsize='small').get_frame().set_edgecolor('none')
ax.set_xlim(1e-5,1e0)
ax.set_ylim(1e-2,1e5)
ax.set_xlabel('$\lambda$ [cm]')
ax.set_ylabel(r'$\kappa^\mathrm{abs,tot}_\nu$ [cm$^2$/g]')

#f.subplots_adjust(
#    bottom=0.171, top=0.94,
#    left=0.15, right=0.97)

f.subplots_adjust(
    bottom=0.09, top=0.94,
    left=0.09, right=0.97)

plt.savefig('figures/fig6_comparison.pdf')